<a href="https://www.kaggle.com/code/gojocloud/gojo-s-first-analysis?scriptVersionId=263915813" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


此处添加一步特征工程，包含IsAlone和FamilySize。先前的整个流程是没有特征工程的，加入后结果更加准确（9.23kaggleTitanic规划）

In [6]:
# ===========================
# 按“五类”重新特征工程
# ===========================

# 1. 客舱等级 → 保留 Pclass 即可，已是 1<2<3 序数，不再动

# 2. 性别 → 独热 1 维
train_data["Sex_male"] = (train_data["Sex"] == "male").astype(int)
test_data["Sex_male"]  = (test_data["Sex"] == "male").astype(int)

# 3. 年龄 → 分 5 段 + 缺失旗标
for df in [train_data, test_data]:
    df["Age_null"] = df["Age"].isnull().astype(int)          # 缺失旗标
    age_fill = df["Age"].fillna(df["Age"].median())
    df["Age_child"]  = (age_fill < 13).astype(int)
    df["Age_teen"]   = ((age_fill >= 13) & (age_fill < 18)).astype(int)
    df["Age_young"]  = ((age_fill >= 18) & (age_fill < 36)).astype(int)
    df["Age_mid"]    = ((age_fill >= 36) & (age_fill < 60)).astype(int)
    df["Age_senior"] = (age_fill >= 60).astype(int)

# 4. 同代亲属（SibSp）→ 0 / 1 / 2 / 3+
for df in [train_data, test_data]:
    df["SibSp_0"]  = (df["SibSp"] == 0).astype(int)
    df["SibSp_1"]  = (df["SibSp"] == 1).astype(int)
    df["SibSp_2"]  = (df["SibSp"] == 2).astype(int)
    df["SibSp_3p"] = (df["SibSp"] >= 3).astype(int)

# 5. 隔代亲属（Parch）→ 0 / 1 / 2 / 3+
for df in [train_data, test_data]:
    df["Parch_0"]  = (df["Parch"] == 0).astype(int)
    df["Parch_1"]  = (df["Parch"] == 1).astype(int)
    df["Parch_2"]  = (df["Parch"] == 2).astype(int)
    df["Parch_3p"] = (df["Parch"] >= 3).astype(int)

# --- 最终特征列表（按你五类思路汇总） -------------
features = ["Pclass",
            "Sex_male",
            "Age_child", "Age_teen", "Age_young", "Age_mid", "Age_senior", "Age_null",
            "SibSp_0", "SibSp_1", "SibSp_2", "SibSp_3p",
            "Parch_0", "Parch_1", "Parch_2", "Parch_3p"]


In [7]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

# 新特征列表（16 维，已全部 0/1）
features = ["Pclass",
            "Sex_male",
            "Age_child", "Age_teen", "Age_young", "Age_mid", "Age_senior", "Age_null",
            "SibSp_0", "SibSp_1", "SibSp_2", "SibSp_3p",
            "Parch_0", "Parch_1", "Parch_2", "Parch_3p"]

# 无需再 get_dummies，已全数值
X = train_data[features]
X_test = test_data[features]

# 建模参数完全不变
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=6,
    min_samples_split=12,
    min_samples_leaf=7,
    random_state=1
)
model.fit(X, y)
predictions = model.predict(X_test)

# 输出结果（一字不改）
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
